# Build a Simple LLM Application with LCEL

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After reading this tutorial, you'll have a high level overview of:

- Using [language models](/docs/concepts/chat_models)

- Using [PromptTemplates](/docs/concepts/prompt_templates) and [OutputParsers](/docs/concepts/output_parsers)

- Using [LangChain Expression Language (LCEL)](/docs/concepts/lcel) to chain components together

- Debugging and tracing your application using [LangSmith](/docs/concepts/#langsmith)

Let's dive in!

## Setup

### Installation

To install LangChain run:

```{=mdx}
import Npm2Yarn from '@theme/Npm2Yarn';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Npm2Yarn>
  langchain @langchain/core
</Npm2Yarn>
```


For more details, see our [Installation guide](/docs/how_to/installation/).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."

# Reduce tracing latency if you are not in a serverless environment
# export LANGCHAIN_CALLBACKS_BACKGROUND=true
```

## Using Language Models

First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`{ model: "gpt-4" }`} />
```

In [ ]:
// @lc-docs-hide-cell
import { ChatOpenAI } from '@langchain/openai';

const model = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

Let's first use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [2]:
import { HumanMessage, SystemMessage } from "@langchain/core/messages"

const messages = [
  new SystemMessage("Translate the following from English into Italian"),
  new HumanMessage("hi!"),
];

await model.invoke(messages)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "ciao!",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "ciao!",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 3, promptTokens: 20, totalTokens: 23 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

If we've enable LangSmith, we can see that this run is logged to LangSmith, and can see the [LangSmith trace](https://smith.langchain.com/public/45f1a650-38fb-41e1-9b61-becc0684f2ce/r)

## OutputParsers

Notice that the response from the model is an `AIMessage`. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [5]:
import { StringOutputParser } from "@langchain/core/output_parsers";

const parser = new StringOutputParser();

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [3]:
const result = await model.invoke(messages)

In [6]:
await parser.invoke(result)

"ciao!"

## Chaining together components with LCEL

We can also "chain" the model to the output parser. This means this output parser will get called with the output from the model. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can create the chain using the `.pipe()` method. The `.pipe()` method is used in LangChain to combine two elements together.

In [7]:
const chain = model.pipe(parser);

In [8]:
await chain.invoke(messages)

"Ciao!"

This is a simple example of using [LangChain Expression Language (LCEL)](/docs/concepts/lcel) to chain together LangChain modules. There are several benefits to this approach, including optimized streaming and tracing support.

If we now look at LangSmith, we can see that the chain has two steps: first the language model is called, then the result of that is passed to the output parser. We can see the [LangSmith trace](https://smith.langchain.com/public/05bec1c1-fc51-4b2c-ab3b-4b63709e4462/r)

## Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model. 

Let's create a PromptTemplate here. It will take in two user variables:

- `language`: The language to translate text into
- `text`: The text to translate

In [9]:
import { ChatPromptTemplate } from "@langchain/core/prompts"

First, let's create a string that we will format to be the system message:

In [10]:
const systemTemplate = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the `systemTemplate` as well as a simpler template for where to put the text

In [11]:
const promptTemplate = ChatPromptTemplate.fromMessages(
  [
    ["system", systemTemplate],
    ["user", "{text}"]
  ]
)

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [13]:
const promptValue = await promptTemplate.invoke({ language: "italian", text: "hi" })

promptValue

ChatPromptValue {
  lc_serializable: true,
  lc_kwargs: {
    messages: [
      SystemMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "Translate the following into italian:",
          additional_kwargs: {},
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "Translate the following into italian:",
        name: undefined,
        additional_kwargs: {},
        response_metadata: {}
      },
      HumanMessage {
        lc_serializable: true,
        lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "hi",
        name: undefined,
        additional_kwargs: {},
        response_metadata: {}
      }
    ]
  },
  lc_namespace: [ "langchain_core", "prompt_values" ],
  messages: [
    SystemMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Translate the following into ita

We can see that it returns a `ChatPromptValue` that consists of two messages. If we want to access the messages directly we do:

In [15]:
promptValue.toChatMessages()

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Translate the following into italian:",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Translate the following into italian:",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]

We can now combine this with the model and the output parser from above. This will chain all three components together.

In [16]:
const llmChain = promptTemplate.pipe(model).pipe(parser);

In [17]:
await llmChain.invoke({ language: "italian", text: "hi" })

"ciao"

If we take a look at the LangSmith trace, we can see all three components show up in the [LangSmith trace](https://smith.langchain.com/public/cef6edcd-39ed-4c1e-86f7-491a1b611aeb/r)

## Conclusion

That's it! In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to parse their outputs, how to create a prompt template, chaining them together with LCEL, and how to get great observability into chains you create with LangSmith.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!

For further reading on the core concepts of LangChain, we've got detailed [Conceptual Guides](/docs/concepts).

If you have more specific questions on these concepts, check out the following sections of the how-to guides:

- [LangChain Expression Language (LCEL)](/docs/how_to/#langchain-expression-language)
- [Prompt templates](/docs/how_to/#prompt-templates)
- [Chat models](/docs/how_to/#chat-models)
- [Output parsers](/docs/how_to/#output-parsers)

And the LangSmith docs:

- [LangSmith](https://docs.smith.langchain.com)